In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df.shape

(404290, 6)

In [8]:
new_df = df.sample(30000)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
363048,363048,492993,492994,What are the best ways to find jobs in the con...,What is the best way to find jobs in the const...,0
128419,128419,47923,184984,What's the story behind Apple's half eaten app...,What is the story behind the half-eaten Apple ...,1
388049,388049,520404,520405,Which season spoils the yield of the harvested...,Which season spoils the yield of harvested crops?,1
206075,206075,309393,262685,Can someone see if you screenshot their Instag...,How do you know when someone takes a screensho...,1
97954,97954,162865,162866,Should I go for the job if I am a mechanical e...,I am Mechanical engineer placed in TCS IT stre...,0


In [9]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
new_df = new_df.dropna(subset=['question1','question2'])

In [11]:
new_df.duplicated().sum()

0

In [12]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
363048,What are the best ways to find jobs in the con...,What is the best way to find jobs in the const...
128419,What's the story behind Apple's half eaten app...,What is the story behind the half-eaten Apple ...
388049,Which season spoils the yield of the harvested...,Which season spoils the yield of harvested crops?
206075,Can someone see if you screenshot their Instag...,How do you know when someone takes a screensho...
97954,Should I go for the job if I am a mechanical e...,I am Mechanical engineer placed in TCS IT stre...


In [13]:
from sklearn.feature_extraction.text import CountVectorizer


questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [14]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)

In [15]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
363048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206075,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
97954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
temp_df.shape

(30000, 6000)

In [17]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [18]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
363048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
388049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
206075,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
97954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
281088,0,0,0,0,0,0,0,0,0,0,...,0,3,0,1,0,0,0,0,0,0
376505,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,1
247864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state=1)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)
accuracy_score(Y_test, Y_pred)

0.7386666666666667

In [26]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, Y_train)
Y_pred = xgb.predict(X_test)
accuracy_score(Y_test, Y_pred)

0.7141666666666666